In [2]:
from tqdm import tqdm
import numpy as np 
import pandas as pd 
import csv
import random
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from _bootstrap import add_project_root_to_path
add_project_root_to_path()

import config
config.seed_everything(42)

2025-06-20 15:11:43.093219: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750432303.302627      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750432303.356198      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv("/kaggle/input/pricerunner-mobiles-20-06-2025/pricerunner_mobile_20_06_2025.csv")
df

,ID,title,vendor,cluster,cluster_label,category,category_label
0,2,apple iphone 8 plus 64 gb spacegrau,2,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
1,3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,3,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
2,4,apple iphone 8 plus 64gb space grey,4,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
3,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,5,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
4,6,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,6,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
...,...,...,...,...,...,...,...
4034,4087,samsung e800,48,1814,Samsung SGH-E800,2612,Mobile Phones
4035,4088,nokia 7600,48,1815,Nokia 7600,2612,Mobile Phones
4036,4089,nokia 1100,48,1816,Nokia 1100,2612,Mobile Phones
4037,4090,nokia 6310i silver,48,1817,Nokia 6310i,2612,Mobile Phones


In [4]:
df.drop_duplicates(subset=["title"], inplace=True)
df

,ID,title,vendor,cluster,cluster_label,category,category_label
0,2,apple iphone 8 plus 64 gb spacegrau,2,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
1,3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,3,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
2,4,apple iphone 8 plus 64gb space grey,4,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
3,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,5,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
5,7,apple iphone 8 plus 64 gb space grey,7,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
...,...,...,...,...,...,...,...
4034,4087,samsung e800,48,1814,Samsung SGH-E800,2612,Mobile Phones
4035,4088,nokia 7600,48,1815,Nokia 7600,2612,Mobile Phones
4036,4089,nokia 1100,48,1816,Nokia 1100,2612,Mobile Phones
4037,4090,nokia 6310i silver,48,1817,Nokia 6310i,2612,Mobile Phones


In [5]:
import re

# Enable tqdm for pandas
tqdm.pandas()

def preprocess_title(title):
    # Convert to lowercase
    title = title.lower()
    
    # Replace dashes with spaces
    title = re.sub(r"-", " ", title)
    
    # Remove all punctuation except dots, ensuring spaces remain
    title = re.sub(r"[^a-zA-Z0-9. ]", "", title)  # Allows letters, numbers, dots, and spaces
    
    # Normalize multiple spaces to a single space
    title = re.sub(r"\s+", " ", title).strip()
    
    return title

# Example usage:
titles = [
    "Apple iPhone-X 256GB - Black",
    "Samsung Galaxy S21 Ultra 5G - Phantom Silver",
    "Sony Bravia-XR 55” 4K TV - 120Hz Refresh Rate!",
    "Price: $199.99 (Special Offer)",
    "Hello-World!!! Test--Title"
]

processed_titles = [preprocess_title(t) for t in titles]
print(processed_titles)

['apple iphone x 256gb black', 'samsung galaxy s21 ultra 5g phantom silver', 'sony bravia xr 55 4k tv 120hz refresh rate', 'price 199.99 special offer', 'hello world test title']


In [6]:
df["title"] = df["title"].apply(preprocess_title)
df

,ID,title,vendor,cluster,cluster_label,category,category_label
0,2,apple iphone 8 plus 64 gb spacegrau,2,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
1,3,apple mq8n2ba iphone 8 plus 64gb 5.5 12mp sim ...,3,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
2,4,apple iphone 8 plus 64gb space grey,4,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
3,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,5,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
5,7,apple iphone 8 plus 64 gb space grey,7,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
...,...,...,...,...,...,...,...
4034,4087,samsung e800,48,1814,Samsung SGH-E800,2612,Mobile Phones
4035,4088,nokia 7600,48,1815,Nokia 7600,2612,Mobile Phones
4036,4089,nokia 1100,48,1816,Nokia 1100,2612,Mobile Phones
4037,4090,nokia 6310i silver,48,1817,Nokia 6310i,2612,Mobile Phones


In [7]:
# Filter clusters with more than one product
cluster_counts = df["cluster"].value_counts()
valid_clusters = cluster_counts[cluster_counts > 1].index
df_pos = df[df["cluster"].isin(valid_clusters)]
df_pos

,ID,title,vendor,cluster,cluster_label,category,category_label
0,2,apple iphone 8 plus 64 gb spacegrau,2,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
1,3,apple mq8n2ba iphone 8 plus 64gb 5.5 12mp sim ...,3,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
2,4,apple iphone 8 plus 64gb space grey,4,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
3,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,5,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
5,7,apple iphone 8 plus 64 gb space grey,7,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
...,...,...,...,...,...,...,...
3987,4040,nokia 6300 sim free mobile phone black,3,1769,Nokia 6300,2612,Mobile Phones
4006,4059,nokia 6131 mobile phone,48,1788,Nokia 6131,2612,Mobile Phones
4007,4060,nokia handy 6131 schwarzsilber ohne branding,3,1788,Nokia 6131,2612,Mobile Phones
4026,4079,motorola razr v3 black,48,1807,Motorola RAZR V3,2612,Mobile Phones


In [8]:
# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Load SBERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')  
def get_sbert_embeddings(df, attribute='title'):
    """Compute SBERT embeddings for each product title."""
    df['embedding'] = df[attribute].apply(lambda x: sbert_model.encode(x))
    return df

def find_positive_pairs(df, num_pos=2):
    """Find positive pairs within each cluster using cosine similarity."""
    pos_pairs = []

    for cluster_id, cluster_data in tqdm(df.groupby('cluster')):
        cluster_embeddings = np.stack(cluster_data['embedding'].values)
        cluster_ids = cluster_data['ID'].values

        # Compute pairwise cosine similarity
        sim_matrix = cosine_similarity(cluster_embeddings)

        for i, current_id in enumerate(cluster_ids):
            sim_scores = list(enumerate(sim_matrix[i]))
            sim_scores = [(cluster_ids[idx], score) for idx, score in sim_scores if idx != i]

            # Sort by similarity
            sim_scores.sort(key=lambda x: x[1])

            # Ensure we have enough candidates
            current_num_pos = min(num_pos, len(sim_scores))
            hard_pos = current_num_pos // 2
            random_pos = current_num_pos - hard_pos

            # Select hard positives from highest similarity scores
            hard_positives = [x[0] for x in sim_scores[:hard_pos]]

            # Select random positives from remaining cluster members
            remaining = sim_scores[hard_pos:]
            random.shuffle(remaining)
            random_positives = [x[0] for x in remaining[:random_pos]]

            # Append product pairs
            for pos_id in hard_positives:
                pos_pairs.append((current_id, pos_id, 'hard_positive'))

            for pos_id in random_positives:
                pos_pairs.append((current_id, pos_id, 'random_positive'))


    return pos_pairs

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
def find_negative_pairs(df, num_neg=2):
    """Find negative pairs by selecting most similar products from different clusters."""
    neg_pairs = []

    # Extract embeddings, IDs, clusters, and vendors
    all_embeddings = np.stack(df['embedding'].values)
    all_ids = df['ID'].values
    all_clusters = df['cluster'].values

    # Compute pairwise cosine similarity
    sim_matrix = cosine_similarity(all_embeddings)

    for i, current_id in tqdm(enumerate(all_ids)):
        current_cluster = all_clusters[i]

        # Get similarities to other products
        sim_scores = [(all_ids[j], all_clusters[j], sim_matrix[i][j]) 
                      for j in range(len(all_ids)) if j != i]

        # Exclude same-cluster products
        neg_candidates = [x for x in sim_scores if x[1] != current_cluster]

        if len(neg_candidates) == 0:
            neg_pairs.append((current_id, [[], []]))  # No negatives available
            continue

        # Sort by similarity (highest similarity is hardest negative)
        neg_candidates.sort(key=lambda x: x[2], reverse=True)

        # Ensure we have enough candidates
        current_num_neg = min(num_neg, len(neg_candidates))
        hard_neg = current_num_neg // 2
        random_neg = current_num_neg - hard_neg

        # Select hard negatives (most similar from a different vendor and cluster)
        hard_negatives = [x[0] for x in neg_candidates[:hard_neg]]

        # Select random negatives (randomly from different clusters and vendors)
        remaining = neg_candidates[hard_neg:]
        random.shuffle(remaining)
        random_negatives = [x[0] for x in remaining[:random_neg]]

        # Append product pairs
        for neg_id in hard_negatives:
            neg_pairs.append((current_id, neg_id, 'hard_negative'))

        for neg_id in random_negatives:
            neg_pairs.append((current_id, neg_id, 'random_negative'))

    return neg_pairs

In [10]:
def create_pairs_dataframe(df, pos_pairs, neg_pairs):
    """Convert positive and negative pairs into a structured DataFrame with titles instead of IDs."""
    pair_data = []

    # Create a lookup dictionary for titles
    id_to_title = df.set_index('ID')['title'].to_dict()

    # Process positive pairs (label = 1)
    for id1, id2, label in pos_pairs:  # Now each entry is (id1, id2, 'hard_positive' or 'random_positive')
        pair_data.append((id1, id_to_title[id1], id2, id_to_title[id2], 1))  # Positive label

    # Process negative pairs (label = 0)
    for id1, id2, label in neg_pairs:
        pair_data.append((id1, id_to_title[id1], id2, id_to_title[id2], 0))  # Negative label

    # Convert to DataFrame
    df_pairs = pd.DataFrame(pair_data, columns=['ID1', 'Title1', 'ID2', 'Title2', 'Label'])
    
    return df_pairs

In [11]:
# Calculate Sentence-BERT embeddings
df = get_sbert_embeddings(df, attribute='title')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
embeddings_array = np.array(df["embedding"])
np.save("embeddings_array.npy", embeddings_array)

In [21]:
print("Loading dataframe...")
df = pd.read_csv("/kaggle/input/pricerunner-mobiles-20-06-2025/pricerunner_mobile_20_06_2025.csv")

print("Preprocessing dataframe...")
df["title"] = df["title"].apply(preprocess_title)

print("Removing duplicate titles...")
df.drop_duplicates(subset=["title"], inplace=True)

print("Filtering clusters with more than 1 product for positive pairs")
cluster_counts = df["cluster"].value_counts()
valid_clusters = cluster_counts[cluster_counts > 1].index

print("Loading SBERT embeddings...")
loaded_embeddings = np.load('/kaggle/input/embeddings-array-20-06-2025/embeddings_array.npy', allow_pickle=True)
df['embedding'] = list(loaded_embeddings)

print("Building positive pairs...")
df_pos = df[df["cluster"].isin(valid_clusters)]
pos_pairs = find_positive_pairs(df_pos, num_pos=2)

print("Building negative pairs...")
neg_pairs = find_negative_pairs(df, num_neg=2)

print("Creating final pairs DataFrame with product titles...")
df_pairs = create_pairs_dataframe(df, pos_pairs, neg_pairs)

Loading dataframe...
Preprocessing dataframe...
Removing duplicate titles...
Filtering clusters with more than 1 product for positive pairs
Loading SBERT embeddings...
Building positive pairs...


100%|██████████| 827/827 [00:00<00:00, 3668.75it/s]


Building negative pairs...


3660it [00:15, 229.47it/s]


Creating final pairs DataFrame with product titles...


In [22]:
df_pairs

,ID1,Title1,ID2,Title2,Label
0,2,apple iphone 8 plus 64 gb spacegrau,7,apple iphone 8 plus 64 gb space grey,1
1,2,apple iphone 8 plus 64 gb spacegrau,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,1
2,3,apple mq8n2ba iphone 8 plus 64gb 5.5 12mp sim ...,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,1
3,3,apple mq8n2ba iphone 8 plus 64gb 5.5 12mp sim ...,18,apple iphone 8 plus 14 cm 5.5 64 gb 12 mp ios ...,1
4,4,apple iphone 8 plus 64gb space grey,20,iphone 8 plus 64gb space grey,1
...,...,...,...,...,...
12087,4089,nokia 1100,3401,microsoft lumia 950,0
12088,4090,nokia 6310i silver,3856,nokia e72 silver,0
12089,4090,nokia 6310i silver,3330,alcatel idol 4s smart poneparent,0
12090,4091,nokia 7210 supernova mobile phones,3941,nokia 7310 supernova mobile phone,0


In [23]:
from sklearn.utils import shuffle

# Shuffle the dataframe
df_pairs = shuffle(df_pairs, random_state=42).reset_index(drop=True)

# Save the final pairs DataFrame
df_pairs.to_csv('/kaggle/working/final_pairs_mobiles_20_06_2025.csv', index=False)
print(f"Final dataset contains {len(df_pairs)} pairs. Saved to CSV.")

Final dataset contains 12092 pairs. Saved to CSV.
